In [1]:
# Required packes
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model

# Input model
modelM=read_sbml_model('iJN678mod.xml')

In [12]:
#Setting medium for autotrophic (CO2 as carbon source)

co2KO=modelM.reactions.get_by_id("EX_co2_e")
photonKO=modelM.reactions.get_by_id("EX_photon_e")
hco3KO=modelM.reactions.get_by_id("EX_hco3_e")

co2KO.bounds=(-3.7,1000.0)
hco3KO.bounds= (-3.7,1000.0)
photonKO.bounds= (-100,0)

mediumAuto = modelM.medium
mediumAuto["EX_glc__D_e"] = 0.0
mediumAuto["EX_hco3_e"] = 0.0
mediumAuto["EX_photon_e"]=54.5
mediumAuto["EX_co2_e"]=3.7
modelM.medium = mediumAuto

modelM.objective="EX_sql_e"
modelM.optimize().objective_value

sql_production = modelM.optimize().objective_value
maximum_yield = sql_production / (-1*(modelM.reactions.EX_co2_e.flux))
print('Maximum productivity =', sql_production, 'mmol/gDW*h')
print('Maximum theoretical yield =', maximum_yield, 'mmol-sql/mmol-co2')

Maximum productivity = 0.12333333333333386 mmol/gDW*h
Maximum theoretical yield = 0.03333333333333347 mmol-sql/mmol-co2


# MVA Pathway

**Reaction 0**

In [ ]:
# Defining the reactions
reaction0 = Reaction('R_MVA0') # 2 Acetyl-Coa => Acetoacetyl-CoA + CoA
reaction0.name = 'acetyl-CoA C-acetyltransferase'
reaction0.subsystem = 'Melavonate Pathway'
#reaction0.lower_bound = 0.  # Set as a default
#reaction0.upper_bound = 1000.  # Set as a default

# Defining the metabolites
aacoa_c = Metabolite(
    id = 'aacoa_c',
    formula='C25H40N7O18P3S',
    name ='Acetoacetyl-CoA',
    compartment ='c')

# Adding the metabolites to the reaction
reaction0.add_metabolites({
    model.metabolites.get_by_id('accoa_c'): -2, # Acetyl-CoA
    aacoa_c: 1,
    model.metabolites.get_by_id('coa_c'): 1, # Coenzyme A
})

reaction0.reaction # This gives a string representation of the reaction 

# Adding the genes of the reactions
reaction0.gene_reaction_rule = '( G_MVA0 )'

# Adding the reactions to the model
model.add_reactions([reaction0])
model.add_boundary(model.metabolites.aacoa_c, type='demand')

**Reaction 1**

In [ ]:
# Defining the reactions
reaction1 = Reaction('R_MVA1') # Acetoacetyl-CoA + H2O + Acetyl-CoA => (S)-3-hydroxy-3-methylglutaryl-CoA + CoA
reaction1.name = 'Hydroxymethylglutaryl-CoA synthase'
reaction1.subsystem = 'Mevalonate pathway'
#reaction1.lower_bound = 0.  # This is the default
#reaction1.upper_bound = 1000.  # This is the default

# Defining the reactions
hmgcoa_c = Metabolite(
    'hmgcoa_c',
    formula='C27H44N7O20P3S',
    name='(S)-3-hydroxy-3-methylglutaryl-CoA',
    compartment='c')

# Defining the reactions
reaction1.add_metabolites({
    model.metabolites.get_by_id('aacoa_c'): -1,
    model.metabolites.get_by_id('h2o_c'): -1,
    model.metabolites.get_by_id('accoa_c'): -1,
    hmgcoa_c: 1,
    model.metabolites.get_by_id('coa_c'): 1,
})

# Adding the reactions to the model
reaction1.reaction # This gives a string representation of the reaction

reaction1.gene_reaction_rule = '( G_MVA1 )'

model.add_reactions([reaction1])
model.add_boundary(model.metabolites.hmgcoa_c, type='demand')

**Reaction 2**

In [ ]:
# Defining the reactions
reaction2 = Reaction('R_MVA2') # MHG-CoA + 2 NADPH + 2H+ => Mevalonate + 2 NADP+ + CoA
reaction2.name = 'hydroxymethylglutaryl-CoA reductase (NADPH)'
reaction2.subsystem = 'Mevalonate pathway'
#reaction2.lower_bound = 0.  # This is the default
#reaction2.upper_bound = 1000.  # This is the default

# Defining the reactions
mva_c = Metabolite(
    'mva_c',
    formula='C6O4H12',
    name='Mevalonate',
    compartment='c')

# Defining the reactions
reaction2.add_metabolites({
    model.metabolites.get_by_id('hmgcoa_c'): -1,
    model.metabolites.get_by_id('nadph_c'): -2,
    model.metabolites.get_by_id('h_c'): -2,
    mva_c: 1,
    model.metabolites.get_by_id('nadp_c'): 2,
    model.metabolites.get_by_id('coa_c'): 1,
})

# Adding the reactions to the model
reaction2.reaction # This gives a string representation of the reaction 

reaction2.gene_reaction_rule = '( G_MVA2 )'

model.add_reactions([reaction2])
model.add_boundary(model.metabolites.mva_c, type='demand')

**Reaction 3**

In [ ]:
# Defining the reactions
reaction3 = Reaction('R_MVA3') # Mevalonate + ATP => Mevalonate-5-phoshphate + ADP
reaction3.name = 'mevalonate kinase '
reaction3.subsystem = 'Mevalonate pathway'
#reaction3.lower_bound = 0.  # This is the default
#reaction3.upper_bound = 1000.  # This is the default

# Defining the reactions
mvap_c = Metabolite(
    'mvap_c',
    formula='C6H13O7P',
    name='Mevalonate-5-phoshphate',
    compartment='c')

# Defining the reactions
reaction3.add_metabolites({
    model.metabolites.get_by_id('mva_c'): -1,
    model.metabolites.get_by_id('atp_c'): -1,
    mvap_c: 1,
    model.metabolites.get_by_id('adp_c'): 1,
})

# Adding the reactions to the model
reaction3.reaction # This gives a string representation of the reaction 

reaction3.gene_reaction_rule = '( G_MVA3 )'

model.add_reactions([reaction3])
model.add_boundary(model.metabolites.mvap_c, type='demand')

**Reaction 4**

In [ ]:
# Defining the reactions
reaction4 = Reaction('R_MVA4') # Mevalonate-5-phosphate + ATP => Mevalonate-5-diphosphate + ADP
reaction4.name = 'phosphomevalonate kinase'
reaction4.subsystem = 'Mevalonate pathway'
#reaction4.lower_bound = 0.  # This is the default
#reaction4.upper_bound = 1000.  # This is the default

# Defining the reactions
mvapp_c = Metabolite(
    'mvapp_c',
    formula='C6H14O10P2',
    name='Mevalonate-5-diphosphate',
    compartment='c')

# Defining the reactions
reaction4.add_metabolites({
    model.metabolites.get_by_id('mvap_c'): -1,
    model.metabolites.get_by_id('atp_c'): -1,
    mvapp_c: 1,
    model.metabolites.get_by_id('adp_c'): 1,
})

# Adding the reactions to the model
reaction4.reaction # This gives a string representation of the reaction 

reaction4.gene_reaction_rule = '( G_MVA4 )'

model.add_reactions([reaction4])
model.add_boundary(model.metabolites.mvapp_c, type='demand')

**Reaction 5**

In [ ]:
# Defining the reactions
reaction5 = Reaction('R_MVA5') # Mevalonate-5-diphosphate + ATP => Isopenthenyl diphosphate + ADP + CO2 + Pi
reaction5.name = 'diphosphomevalonate decarboxylase'
reaction5.subsystem = 'Mevalonate pathway'
#reaction5.lower_bound = 0.  # This is the default
#reaction5.upper_bound = 1000.  # This is the default

# Defining the reactions
ipdp_c = Metabolite(
    'ipdp_c',
    formula='C5H12O7P2',
    name='Isopenthenyl diphosphate',
    compartment='c')

# Defining the reactions
reaction5.add_metabolites({
    model.metabolites.get_by_id('mvapp_c'): -1,
    model.metabolites.get_by_id('atp_c'): -1,
    ipdp_c: 1,
    model.metabolites.get_by_id('adp_c'): 1,
    model.metabolites.get_by_id('pi_c'): 1,
    model.metabolites.get_by_id('co2_c'): 1,
})

# Adding the reactions to the model
reaction5.reaction # This gives a string representation of the reaction 

reaction5.gene_reaction_rule = '( G_MVA5 )'

model.add_reactions([reaction5])
model.add_boundary(model.metabolites.ipdp_c, type='demand')

**Reaction 6**

In [ ]:
# Defining the reactions
reaction6 = Reaction('R_MVA6') # Geranyl diphosphate + Isopenthenyl diphosphate => 2-trans,6-trans-Farnesyl diphosphate + diphosphate 
reaction6.name = 'farnesyl diphosphate synthase '
reaction6.subsystem = 'Mevalonate pathway'
#reaction6.lower_bound = 0.  # This is the default
#reaction6.upper_bound = 1000.  # This is the default

# Defining the reactions
frdp_c = Metabolite(
    'frdp_c',
    formula='C15H28O7P2',
    name='2-trans,6-trans-Farnesyl diphosphate',
    compartment='c')

# Defining the reactions
reaction6.add_metabolites({
    model.metabolites.get_by_id('grdp_c'): -1,
    model.metabolites.get_by_id('ipdp_c'): -1,
    frdp_c: 1,
    model.metabolites.get_by_id('ppi_c'): 1,
})

# Adding the reactions to the model
reaction6.reaction # This gives a string representation of the reaction 

reaction6.gene_reaction_rule = '( G_MVA6 )'

model.add_reactions([reaction6])
model.add_boundary(model.metabolites.frdp_c, type='demand')

**Reaction 7**

In [ ]:
# Defining the reactions
reaction7 = Reaction('R_MVA7') # (2E,6E)-farnesyl diphosphate => presqualene diphosphate +  diphosphate
reaction7.name = 'farnesyl-diphosphate farnesyltransferase'
reaction7.subsystem = 'Mevalonate pathway'
#reaction7.lower_bound = 0.  # This is the default
#reaction7.upper_bound = 1000.  # This is the default

# Defining the reactions
psql_c = Metabolite(
    'psql_c',
    formula='C30H52O7P2',
    name='Presqualene',
    compartment='c')

# Defining the reactions
reaction7.add_metabolites({
    model.metabolites.get_by_id('frdp_c'): -1,
    psql_c: 1,
    model.metabolites.get_by_id('ppi_c'): 1,
})

# Adding the reactions to the model
reaction7.reaction # This gives a string representation of the reaction 

reaction7.gene_reaction_rule = '( G_MVA7 )'

model.add_reactions([reaction7])
model.add_boundary(model.metabolites.psql_c, type='demand')

**Reaction 8**

In [ ]:
# Defining the reactions
reaction8 = Reaction('R_MVA8') # Presqualene diphosphate + NADPH + H+ <=> Squalene + diphosphate + NADP+
reaction8.name = 'farnesyl-diphosphate farnesyltransferase'
reaction8.subsystem = 'Mevalonate pathway'
#reaction8.lower_bound = 0.  # This is the default
#reaction8.upper_bound = 1000.  # This is the default

# Defining the reactions
sql_c = Metabolite(
    'sql_c',
    formula='C30H50',
    name='Squalene',
    compartment='c')

# Defining the reactions
reaction8.add_metabolites({
    model.metabolites.get_by_id('psql_c'): -1,
    model.metabolites.get_by_id('nadph_c'): -1,
    model.metabolites.get_by_id('h_c'): -1,
    sql_c: 1,
    model.metabolites.get_by_id('ppi_c'): 1,
    model.metabolites.get_by_id('nadp_c'):1
})

# Adding the reactions to the model
reaction8.reaction # This gives a string representation of the reaction 

reaction8.gene_reaction_rule = '( G_MVA8 )'

model.add_reactions([reaction8])
model.add_boundary(model.metabolites.sql_c, type='demand')

In [ ]:
# Generate new .xml model file with the added reactions
from cobra.io import write_sbml_model, validate_sbml_model
write_sbml_model(model, filename='iJN678mm.xml')
report = validate_sbml_model(filename='iJN678mm.xml')

print(report)